In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from sklearn.model_selection import train_test_split

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



In [2]:
X_small = pd.read_csv('/kaggle/input/playground-series-s5e2/train.csv', index_col= 'id')
X_test = pd.read_csv('/kaggle/input/playground-series-s5e2/test.csv', index_col= 'id')
X_extra = pd.read_csv('/kaggle/input/playground-series-s5e2/training_extra.csv', index_col= 'id')

X_full = pd.concat([X_small, X_extra], ignore_index=True)

# Remove rows with missing values for price
X_full.dropna(axis=0,subset=['Price'],inplace=True)

# Remove rows with missing values for all
#X_full.dropna(axis=0,subset=['Price','Brand','Material','Size', 'Compartments', 'Laptop Compartment',
#                            'Waterproof','Style', 'Color', 'Weight Capacity (kg)'],inplace=True)

# Seperate Targets
y = X_full.Price
X_full.drop(['Price'],inplace=True, axis=1)


X_full.isna().sum()




Brand                   126758
Material                110962
Size                     87785
Compartments                 0
Laptop Compartment       98533
Waterproof               94324
Style                   104180
Color                   133617
Weight Capacity (kg)      1808
dtype: int64

In [3]:
X_full.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3994318 entries, 0 to 3994317
Data columns (total 9 columns):
 #   Column                Dtype  
---  ------                -----  
 0   Brand                 object 
 1   Material              object 
 2   Size                  object 
 3   Compartments          float64
 4   Laptop Compartment    object 
 5   Waterproof            object 
 6   Style                 object 
 7   Color                 object 
 8   Weight Capacity (kg)  float64
dtypes: float64(2), object(7)
memory usage: 274.3+ MB


In [4]:
# Break validiation set from train data
X_train, X_valid, y_train, y_valid = train_test_split(X_full, y, train_size=1,test_size=0.1)


In [5]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder


# "Cardinality" means the number of unique values in a column
# Select categorical columns with relatively low cardinality (convenient but arbitrary)
categorical_cols = [cname for cname in X_train.columns if X_train[cname].nunique() < 10 and 
                        X_train[cname].dtype == "object"]

# Select numerical columns
numerical_cols = [cname for cname in X_train.columns if X_train[cname].dtype in ['int64', 'float64']]

# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='median')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])
preprocessor


ColumnTransformer(transformers=[('num', SimpleImputer(strategy='median'),
                                 ['Compartments', 'Weight Capacity (kg)']),
                                ('cat',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehot',
                                                  OneHotEncoder(handle_unknown='ignore'))]),
                                 ['Brand', 'Material', 'Size',
                                  'Laptop Compartment', 'Waterproof', 'Style',
                                  'Color'])])

In [6]:
from xgboost import XGBRegressor

xgb_model = XGBRegressor(n_estimators=1500, learning_rate=0.05)

In [7]:
import sklearn

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', xgb_model)
                             ])

# 1. Fit the preprocessor on X_train:
preprocessor.fit(X_train)

# 2. Transform the validation set:
X_valid_transformed = preprocessor.transform(X_valid)

# Preprocessing of training data, fit model 
my_pipeline.fit(
    X_train, y_train,
    model__early_stopping_rounds=5,
    model__eval_set=[(X_valid_transformed, y_valid)],
    model__verbose=False
)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model 
score = np.sqrt(sklearn.metrics.mean_squared_error(preds, y_valid))
print('RMSE:', score)

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(


RMSE: 68.33500723682855


In [8]:
# Predict on test set
test_pred = my_pipeline.predict(X_test)

output = pd.DataFrame({'id': X_test.index,
                      'Price': test_pred})

output.to_csv('submission.csv', index=False)

In [9]:
!pip3 freeze > requirements.txt  